In [3]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# Created at 

# =============================================================================
# DOCS
# =============================================================================

"""train_data
"""

# =============================================================================
# IMPORTS
# =============================================================================

from IPython.core import debugger as ipdb
import os

import numpy as np
import pandas as pd
from itertools import chain

import sys
import time
import trainer_alpha
import torch
from torch import optim
from torch.utils.data import TensorDataset, DataLoader

import os
from pathlib import Path
import paus_sexp as paus_data

from matplotlib import pyplot as plt
import utils

import local_settings
import loaders

sys.path.append('../code')
# pretrain = True #False

"""Constants
"""
# alpha
alpha = 0.8

# train set size
Ntrain = 8000 #'all'   # Ntrain would have to be an input in the get_loaders function.

# train set size
Ntrain = 100 #'all'

# verpretrain
verpretrain = 3

# paus data
df_galcat = local_settings.galcat
df_cosmos = local_settings.cosmos
df_fa = local_settings.df_fa

data = paus_data.paus(True, df_galcat, df_fa, df_cosmos)
flux, flux_err, fmes, vinv, isnan, zbin, ref_id = data

version = 2
output_dir = Path(local_settings.redux_path) / str(version)

# Other values collided with importing the code in a notebook...
catnr = 0 #if len(sys.argv) == 1 else int(sys.argv[1])
inds_all = np.loadtxt(local_settings.inds_large_v1_path)



def train(pretrain_v_path, ifold, **config):
    """Train the networks for one fold.

    :param pretrain_v_path: 

    :type pretrain_v_path: 

    :param ifold: ---

    :type ifold:--

    :param config:

    :type config:

    :return:

    :rtype:

    """
    #: verpretrain: is the version pretrain
    verpretrain = config['verpretrain']

    #: pretrain is pretrain
    pretrain = config['pretrain']

    # part is ?
    part = 'mdn' if use_mdn else 'normal'

    # Where to find the pretrained files.
    path_base = pretrain_v_path + f'{verpretrain}' + '_{}_' + part + '.pt'

    # Indices of the selected sources with flow information and 
    # ¿¿¿config['catnr']???.
    inds = inds_all[config['catnr']][:len(flux)]
    
    #
    enc, dec, net_pz = utils.get_nets(path_base, use_mdn, pretrain)
    train_dl, test_dl, _ = loaders.get_loaders(ifold, inds, data, Ntrain)

    K = (enc, dec, net_pz, train_dl, test_dl, use_mdn, config['alpha'], config['Nexp'], \
         config['keep_last'])

    def params():
        return chain(enc.parameters(), dec.parameters(), net_pz.parameters())
   
    wd = 1e-4
    if True: #False: #False: #False: #True: #False: #True: #True: #False: #True: #False: #False: #True: #False: #True: #pretrain:
        optimizer = optim.Adam(params(), lr=1e-3, weight_decay=wd)
        trainer_alpha.train(optimizer, 100, *K)

    print('main train function...')
    optimizer = optim.Adam(params(), lr=1e-4, weight_decay=wd)
    trainer_alpha.train(optimizer, 200, *K)
    
    optimizer = optim.Adam(params(), lr=1e-5, weight_decay=wd)
    trainer_alpha.train(optimizer, 200, *K)

    optimizer = optim.Adam(params(), lr=1e-6, weight_decay=wd)
    trainer_alpha.train(optimizer, 200, *K)
    
    return enc, dec, net_pz


def pz_fold(ifold, inds, out_fmt, use_mdn):
    """Estimate the photo-z for one fold."""
    
    # Load network..
    #model_dir = Path('/nfs/astro/eriksen/deepz/encmodels_data')
    #path = str(model_dir/ f'{ifold}.pt')
    
    # Loading the networks...
    net_base_path = out_fmt.format(ifold=ifold, net='{}')
    enc, dec, net_pz = utils.get_nets(str(net_base_path), use_mdn)
    enc.eval(), dec.eval(), net_pz.eval()
    
    _, test_dl, zbin_test = loaders.get_loaders(ifold, inds, data, Ntrain)

   
    assert isinstance(inds, torch.Tensor), 'This is required...'
 
    # OK, this needs some improvement...
    L = []
    for Bflux, Bfmes, Bvinv, Bisnan, Bzbin in test_dl:
        Bcoadd, touse = trainer_sexp.get_coadd(Bflux, Bfmes, Bvinv, Bisnan, alpha=1)
        assert touse.all()
            
        # Testing training augmentation.            
        feat = enc(Bcoadd)
        Binput = torch.cat([Bcoadd, feat], 1)
        pred = net_pz(Binput)
        
        zb_part = 0.001*pred.argmax(1).type(torch.float)
        L.append(zb_part)

    zb_fold = torch.cat(L).detach().cpu().numpy()
    zs_fold = 0.001*zbin_test.type(torch.float)

    refid_fold = ref_id[inds == ifold]
    D = {'zs': zs_fold, 'zb': zb_fold, 'ref_id': refid_fold}
    
    part = pd.DataFrame(D)
    part['ifold'] = ifold
    #part = np.vstack([zs_fold, zb_fold]).T

    return part


def train_all(pretrain_v_path, **config):
    """Train all the folds."""
   
    out_fmt = config['out_fmt']
    for ifold in range(5):
        test_path = str(out_fmt.format(net='enc', ifold=ifold))
        if os.path.exists(test_path):
            print('Alread run, skipping:', test_path)
            continue
            
        print('Running for:', ifold)
        t1 = time.time()
        enc, dec, net_pz = train(pretrain_v_path, ifold, **config)
        enc.eval()
        dec.eval()
        net_pz.train()
        
        print('time', time.time() - t1)
       
        #path = str(model_dir/ f'{ifold}.pt')
        torch.save(enc.state_dict(), str(out_fmt.format(net='enc', ifold=ifold)))
        torch.save(dec.state_dict(), str(out_fmt.format(net='dec', ifold=ifold)))
        torch.save(net_pz.state_dict(),  str(out_fmt.format(net='netpz', ifold=ifold)))
                   
def photoz_all(**config):
    """Run the photo-z for all folds."""
                   
    L = []
    inds = inds_all[config['catnr']][:len(flux)]

    inds = torch.Tensor(inds) # Inds_all should be a tensor in the first place.
    for ifold in range(5):
        L.append(pz_fold(ifold, inds, config['out_fmt'], config['use_mdn']))
        
    df = pd.concat(L)
    df = df.set_index('ref_id')

    return df


import trainer_sexp

#model_dir = Path('models/v7')
# Where we store the models based on the data...
version = 9
sim = 'fsps'

def gen_conf():
    for catnr in range(10):
        for keep_last in [True]:
            for alpha in [0.8]:
#[0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
                yield catnr, keep_last, alpha

# This code for running different configurations has been moved out into a notebook.
if True: #True: #False: #False: #False: #True: #False:
    #verpretrain = 3
    #Ntrain = 8000
    #catnr = 0

    use_mdn = True
    model_dir = Path('/data/astro/scratch/idazaper/deepz/redux/train') / str(version)

    verpretrain = 8
    Ntrain = 'all'
    keep_last = False

    label = 'march11'

    for catnr, keep_last, alpha in gen_conf():
           #for Ntrain in ['all']:
        pretrain = False if verpretrain == 'no' else True
        config = {'verpretrain': verpretrain, 'Ntrain': Ntrain, 'catnr': catnr, 'use_mdn': use_mdn,
                  'Ntrain': Ntrain, 'pretrain': pretrain, 'keep_last': keep_last}

        config['Nexp'] = 0
        config['alpha'] = alpha
 
            #    config['output_dir'] = output_dir
#        out_fmt = 'pre{verpretrain}_alpha{alpha}_keep{keep_last}_catnr{catnr}'.format(**config)
        out_fmt = '{net}_'+label+'_ifold{ifold}.pt'
        out_fmt = str(model_dir / out_fmt)

        config['out_fmt'] = out_fmt

        print('To store at:')
        print(out_fmt)

        path = '/data/astro/scratch/idazaper/deepz/'
        pretrain_v_path = os.path.join(path, 'redux/pretrain/v')
        train_all(pretrain_v_path, **config) 

        pz = photoz_all(**config)
        pz['dx'] = (pz.zb - pz.zs) / (1 + pz.zs)

        sig68 = 0.5*(pz.dx.quantile(0.84) - pz.dx.quantile(0.16))
        print('keep_last', keep_last, 'alpha', alpha, 'sig68', sig68)

        fname = f'{label}'+'_catnr{catnr}.csv'.format(**config)
        path_out = model_dir / fname

        pz.to_csv(path_out) 

        # By now we only want to run one catalogue.

        break
    #cat_out = str(output_dir / f'pzcat_{catnr}_mdn.csv') #'/nfs/pic.es/user/e/eriksen/papers/deepz/sims/cats/pzcat_v65_mdn.csv'
    #pz.to_csv(cat_out)


SyntaxError: invalid syntax (utils.py, line 19)

In [4]:
import torch
data = [[[1, 2],[3, 0]], [[1, 2],[4, 0]]]
x_data = torch.tensor(data)
0.8*torch.ones_like(x_data)

tensor([[[0.8000, 0.8000],
         [0.8000, 0.8000]],

        [[0.8000, 0.8000],
         [0.8000, 0.8000]]])

In [7]:
assert isinstance(data, torch.Tensor)

SyntaxError: invalid syntax (615174148.py, line 1)

In [21]:
x_data.sum(2)

tensor([[3, 3],
        [3, 4]])

In [22]:
torch.cat?

Docstring:
cat(tensors, dim=0, *, out=None) -> Tensor

Concatenates the given sequence of :attr:`seq` tensors in the given dimension.
All tensors must either have the same shape (except in the concatenating
dimension) or be empty.

:func:`torch.cat` can be seen as an inverse operation for :func:`torch.split`
and :func:`torch.chunk`.

:func:`torch.cat` can be best understood via examples.

Args:
    tensors (sequence of Tensors): any python sequence of tensors of the same type.
        Non-empty tensors provided must have the same shape, except in the
        cat dimension.
    dim (int, optional): the dimension over which the tensors are concatenated

Keyword args:
    out (Tensor, optional): the output tensor.

Example::

    >>> x = torch.randn(2, 3)
    >>> x
    tensor([[ 0.6580, -1.0969, -0.4614],
            [-0.1034, -0.5790,  0.1497]])
    >>> torch.cat((x, x, x), 0)
    tensor([[ 0.6580, -1.0969, -0.4614],
            [-0.1034, -0.5790,  0.1497],
            [ 0.6580, -1.0969,

In [ ]:

#model_dir = Path('models/v7')
# Where we store the models based on the data...
version = 9
sim = 'fsps'

def gen_conf():
    for catnr in range(10):
        for keep_last in [True]:
            for alpha in [0.8]:
#[0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
                yield catnr, keep_last, alpha

if True: #True: #False: #False: #False: #True: #False:
    #verpretrain = 3
    #Ntrain = 8000
    #catnr = 0

    use_mdn = True
    model_dir = Path('/data/astro/scratch/idazaper/deepz/redux/train') / str(version)

    verpretrain = 8
    Ntrain = 'all'
    keep_last = False

    label = 'march11'

    for catnr, keep_last, alpha in gen_conf():
           #for Ntrain in ['all']:
        pretrain = False if verpretrain == 'no' else True
        config = {'verpretrain': verpretrain, 'Ntrain': Ntrain, 'catnr': catnr, 'use_mdn': use_mdn,
                  'Ntrain': Ntrain, 'pretrain': pretrain, 'keep_last': keep_last}

        config['Nexp'] = 0
        config['alpha'] = alpha
 
        out_fmt = '{net}_'+label+'_ifold{ifold}.pt'
        out_fmt = str(model_dir / out_fmt)

        config['out_fmt'] = out_fmt

        print('To store at:')
        print(out_fmt)

        path = '/data/astro/scratch/idazaper/deepz/'
        pretrain_v_path = os.path.join(path, 'redux/pretrain/v') #### AHHHH DIO CÓMO LO AGREGO?
        train_all(pretrain_v_path, **config) 

        pz = photoz_all(**config)
        pz['dx'] = (pz.zb - pz.zs) / (1 + pz.zs)

        sig68 = 0.5*(pz.dx.quantile(0.84) - pz.dx.quantile(0.16))
        print('keep_last', keep_last, 'alpha', alpha, 'sig68', sig68)

        fname = f'{label}'+'_catnr{catnr}.csv'.format(**config)
        path_out = model_dir / fname

        pz.to_csv(path_out) 

        break


In [10]:

from IPython.core import debugger as ipdb
import os

import numpy as np
import pandas as pd
from itertools import chain

import sys
import time
import trainer_alpha
import torch
from torch import optim
from torch.utils.data import TensorDataset, DataLoader

import os
from pathlib import Path
import paus_sexp as paus_data

from matplotlib import pyplot as plt
import utils

import local_settings
import loaders

loaded path /nfs/pic.es/user/i/idazaper/deepz/deepz/arch_mdn.py


In [13]:
version = 9
label = 'march11'
model_dir = Path('/data/astro/scratch/idazaper/deepz/redux/train') / str(version)
out_fmt = '{net}_'+label+'_ifold{ifold}.pt'
out_fmt = str(model_dir / out_fmt)



print('To store at:')
print(out_fmt)


for ifold in range(5):
    test_path = str(out_fmt.format(net='enc', ifold=ifold))
    print(test_path)

To store at:
/data/astro/scratch/idazaper/deepz/redux/train/9/{net}_march11_ifold{ifold}.pt
/data/astro/scratch/idazaper/deepz/redux/train/9/enc_march11_ifold0.pt
/data/astro/scratch/idazaper/deepz/redux/train/9/enc_march11_ifold1.pt
/data/astro/scratch/idazaper/deepz/redux/train/9/enc_march11_ifold2.pt
/data/astro/scratch/idazaper/deepz/redux/train/9/enc_march11_ifold3.pt
/data/astro/scratch/idazaper/deepz/redux/train/9/enc_march11_ifold4.pt


In [1]:

def gen_conf():
    for catnr in range(10):
        for keep_last in [True]:
            for alpha in [0.8]:
#[0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
                yield catnr, keep_last, alpha

In [2]:
catnr, keep_last, alpha

NameError: name 'catnr' is not defined

In [3]:
for catnr, keep_last, alpha in gen_conf():
    print(catnr, keep_last, alpha)

0 True 0.8
1 True 0.8
2 True 0.8
3 True 0.8
4 True 0.8
5 True 0.8
6 True 0.8
7 True 0.8
8 True 0.8
9 True 0.8


In [ ]:
#######################################################################
"""Input
"""
# paus data
df_galcat = local_settings.galcat
df_cosmos = local_settings.cosmos
df_fa = local_settings.df_fa
data = paus_data.paus(True, df_galcat, df_fa, df_cosmos)

# index
inds_all = local_settings.inds_all

"""Output
"""
version = 2
output_dir = local_settings.output_dir

"""Constants
"""
# alpha
alpha = 0.8

# train set size
Ntrain = 100 #'all'

# verpretrain
verpretrain = 3

# Other values collided with importing the code in a notebook...
catnr = 0 #if len(sys.argv) == 1 else int(sys.argv[1])


"""Config
""""